In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
#Mostrar las primeras filas del conjunto de datos para inspección
print(dataset.head()) # Change 'data' to 'dataset'

In [ ]:
from sklearn.preprocessing import OneHotEncoder # Import OneHotEncoder
categorical_features = ['sex', 'smoker', 'region']
# Replace 'sparse' with 'sparse_output'
encoder = OneHotEncoder(drop='first', sparse_output=False)
categorical_data = encoder.fit_transform(dataset[categorical_features]) # Change 'data' to 'dataset'

In [ ]:
# Escalar las características numéricas (como 'age', 'bmi', 'children')
from sklearn.preprocessing import StandardScaler # Import StandardScaler
categorical_features = ['sex', 'smoker', 'region']
numerical_features = ['age', 'bmi', 'children']
scaler = StandardScaler()
numerical_data = scaler.fit_transform(dataset[numerical_features]) # Change 'data' to 'dataset'

In [ ]:
# Convertir variables categóricas a datos numéricos
categorical_columns = ['sex', 'smoker', 'region']
dataset = pd.get_dummies(dataset, columns=categorical_columns, drop_first=True)


In [ ]:
# Separar las características (X) y la variable objetivo (y)

charges_column ='expenses'
if charges_column in dataset.columns:
    X = dataset.drop(columns=[charges_column], axis=1).values
    y = dataset[charges_column].values


In [ ]:
# Escalado de características
scaler = StandardScaler()
numerical_features = ['age', 'bmi', 'children']


numerical_features_idx = [dataset.columns.get_loc(col) for col in numerical_features if col in dataset.columns]
X[:, numerical_features_idx] = scaler.fit_transform(X[:, numerical_features_idx])

In [ ]:
# Dividir el conjunto de datos en entrenamiento (80%) y prueba (20%)
from sklearn.model_selection import train_test_split # Import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32)
train_labels = y_train

test_dataset = tf.data.Dataset.from_tensor_slices(X_test).batch(32)
test_labels = y_test

In [ ]:
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])
model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mse'])

history = model.fit(X_train, y_train, epochs=100, validation_split=0.2, verbose=1)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(32)


model.fit(train_dataset, epochs=100, validation_data=val_dataset, verbose=0)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
